In [1]:
## Import all packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, NavigableString
from urllib.request import urlopen
import os, re
import numpy as np
import pubchempy as pcp
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from mordred import Calculator, descriptors
trainingSetDF = pd.read_excel("DrugInformation_ImmuneModPaper.xlsx")
###Drop duplicates (certain drugs listed multiple times with slightly different smiles)
trainingSetDF = trainingSetDF.drop_duplicates(subset = "CHEMBL_ID")
##Remove compounds without ChEMBL IDs
trainingSetDF = trainingSetDF[~pd.isna(trainingSetDF.CHEMBL_ID)]
##Using sturcture links_1 because chembl_ID was written differently.
drugBankDF = pd.read_csv("structure links_1.csv")
##Remove compounds without ChEMBL IDs
drugBankDF = drugBankDF[~pd.isna(drugBankDF.CHEMBL_ID)]
##How many drugs overlap between the training set and drugbank?
overlap = len(set(trainingSetDF.CHEMBL_ID.unique()).intersection(set(drugBankDF.CHEMBL_ID.unique())))
print(overlap)
##Merge
combinedDF = pd.merge(trainingSetDF, drugBankDF, on= "CHEMBL_ID", how = "inner")
assert combinedDF.shape[0] == overlap

747


In [2]:
## molecular calculator
calc = Calculator(descriptors, ignore_3D = True)

In [3]:
#combinedDF['SMILES_x']

In [3]:
def m_calculator(dataframe):
    calc = Calculator(descriptors, ignore_3D = True)
    mols = [Chem.MolFromSmiles(smi) for smi in dataframe['SMILES_x']]
    calDF = calc.pandas(mols)
    return (calDF)

In [4]:
# use SMILES to calculate 
mols = [Chem.MolFromSmiles(smi) for smi in combinedDF['SMILES_x']]
## Create a dataframe after calculation
calDF = calc.pandas(mols)

 28%|██▊       | 208/747 [00:32<01:54,  4.71it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 41%|████      | 307/747 [00:48<03:03,  2.40it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 81%|████████  | 605/747 [01:30<00:25,  5.54it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 747/747 [01:46<00:00,  6.99it/s]


In [5]:
## Adding CHEMBL_ID as index
calDF = calDF.set_index(x for x in combinedDF['CHEMBL_ID'])

In [6]:
calDF.shape

(747, 1613)

In [7]:
## Change data types to float
calDFFloat = pd.DataFrame(calDF, dtype='float')

In [14]:
calDFFloat.shape

(747, 1613)

In [8]:
## Drop columns with more than 5% missing values.
calDF_test = calDFFloat[calDFFloat.columns[calDFFloat.isnull().mean() < .05]]
calDF_test.shape

(747, 1392)

In [9]:
## Check how many columns have 0 std.
std_vals = calDF_test.std()

In [17]:
count = 0
for i in std_vals:
    
    if i == 0:
        count += 1
print(count)

134


In [10]:
## Remove std = 0 columns
calDF_ST = calDF_test[calDF_test.columns[calDF_test.std() != 0]]

In [11]:
calDF_ST.shape

(747, 1258)

In [40]:
combinedDFF = pd.merge(combinedDF, calDF_ST, on= "CHEMBL_ID", how = "inner")

In [41]:
combinedDFF.head()

,SMILES_x,CHEMSPIDER_ID,CHEMBL_ID,CHEMBL_KNOWN_DRUG,CHEMBL_NAME,CHEMBL_ALTERNATE_NAME,CLINICALTRIALS_DB,CONCENTRATION_uM,DrugBank ID,Name,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,BrC1=CN([C@@H]2O[C@H](CO)[C@@H](O)C2)C(=O)NC1=O,5813.0,CHEMBL222280,No,BROXURIDINE,SID50105447|SID17389223|SID90341500|SID5646351...,1,10,DB12028,Broxuridine,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
1,Brc1c2nccnc2ccc1NC=1NCCN=1,2341.0,CHEMBL844,Yes,BRIMONIDINE,Alphagan|Brimonidine|UK-14304|Alphagan-P|SID10...,256,10,DB00484,Brimonidine,...,9.736193,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000
2,Brc1cc(cc(Br)c1O)C(=O)c1c2c(oc1CC)cccc2,2243.0,CHEMBL388590,No,BENZBROMARONE,SID11110651|SID26664708|SID24424558|SID5010399...,2,10,DB12319,Benzbromarone,...,10.136938,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111
3,Brc1ccc(cc1)[C@H](CCN(C)C)c1ncccc1,16068.0,CHEMBL1201287,Yes,DEXBROMPHENIRAMINE,SID90341837|Dexbrompheniramine|SID29216402|,0,10,DB00405,Dexbrompheniramine,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
4,Brc1ccccc1C[N+](CC)(C)C,2337.0,CHEMBL1199080,Yes,BRETYLIUM,SID50113155|Bretylium|SID50113154|,0,17,DB01158,Bretylium,...,9.256938,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444


In [12]:
calDF_ST.insert(0,"CHEMBL_ID",[x for x in combinedDF['CHEMBL_ID']])

In [13]:
calDF_ST.head()

,CHEMBL_ID,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
CHEMBL222280,CHEMBL222280,13.003721,11.805899,0.0,0.0,21.163013,2.430127,4.790842,21.163013,1.244883,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
CHEMBL844,CHEMBL844,13.423098,11.209659,0.0,3.0,22.818993,2.412409,4.818966,22.818993,1.342294,...,9.736193,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000
CHEMBL388590,CHEMBL388590,17.125041,14.795147,0.0,0.0,28.373722,2.499388,4.899958,28.373722,1.289715,...,10.136938,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111
CHEMBL1201287,CHEMBL1201287,14.389425,12.216867,0.0,1.0,24.221083,2.349628,4.699256,24.221083,1.274794,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
CHEMBL1199080,CHEMBL1199080,9.579175,9.100648,0.0,1.0,15.610374,2.304315,4.608630,15.610374,1.200798,...,9.256938,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444


In [14]:
def retrieveBioassayInfo(cid):
    aid = []
    score = []
    gene = []
    act = []
    try:
        page = urlopen('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'+cid +'/assaysummary/XML')
        soup = BeautifulSoup(page, "html.parser")
        colNames = soup.find("columns").find_all("column")
        aidIndex = -1
        activityIndex = -1
        for i, col in enumerate(colNames):
            if col.text == "AID":
                aidIndex = i
            elif col.text == "Activity Value [uM]":
                scoreIndex = i
            elif col.text == "Bioactivity Outcome":
                activityIndex = i
            elif col.text == "Target GeneID":
                geneIndex = i
        for row in soup.find_all("row"):
            values= row.find_all("cell")
            if values[geneIndex].text !=  '':
                score.append(values[scoreIndex].text)
                gene.append(values[geneIndex].text)
                aid.append(values[aidIndex].text)
                act.append(values[activityIndex].text)
    except Exception as e: 
        pass
    return(aid, act, gene, score)

In [15]:
def collectDrugActiveTar(cid):
    (aid, act, gene, score) = retrieveBioassayInfo(cid)
    activeIndices = [i for i, x in enumerate(act) if x == "Active"]
    cidList = [cid]*len(activeIndices)
    df = pd.DataFrame({'cid':cidList,
                   'aid':[x for i, x in enumerate(aid) if i in activeIndices],
                    'score':[x for i, x in enumerate(score) if i in activeIndices], 
                    'entrezID':[x for i, x in enumerate(gene) if i in activeIndices]})
    return(df)

In [16]:
def createFullTargetDF(drugData):
    """
    This method creates a full df with all pubchem targets
    :param: a drug data frame with one column listed with a drug's cid and a second with drug id
    :return: data frame with all bioassay target information
    """
    fullTargetDF = pd.DataFrame(columns=['cid','aid','score',
                                         'entrezID', "drug_id"])
    for i, row in tqdm(drugData.iterrows(), total=drugData.shape[0]):
        ##Collect drug targets 
        pubChemDF = collectDrugActiveTar(row["PubCHEM"])
        
        ##get drug id information
        nameDF = pd.DataFrame({"drug_id": [row["drug_id"]]*pubChemDF.shape[0]})
        
        ##combine all drug information
        drugDF = pd.concat([pubChemDF,nameDF],axis=1)
        
        fullTargetDF = pd.concat([fullTargetDF, drugDF])
    ##write out
    return(fullTargetDF)

In [17]:
# Get CID and drugname from combined DF
cid_drug = combinedDF[['PubChem Compound ID','Name']]

In [18]:
cid_drug = cid_drug.rename(columns={"PubChem Compound ID": "PubCHEM", "Name": "drug_id"})

In [20]:
## checking data types for each columns
cid_drug.info()
cid_drug = cid_drug.astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747 entries, 0 to 746
Data columns (total 2 columns):
PubCHEM    720 non-null float64
drug_id    747 non-null object
dtypes: float64(1), object(1)
memory usage: 17.5+ KB


In [73]:
## drop columns with missing PubChemCompund ID
cid_drug = cid_drug.dropna()
cid_drug.shape
cid_drug.astype(str)
## change it to integer to remove decimal points
cid_drug = cid_drug.astype({"PubCHEM": int, "drug_id": object})
cid_drug.info()
## change it to string type to use it on created function
cid_drug.astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 746
Data columns (total 2 columns):
PubCHEM    720 non-null int64
drug_id    720 non-null object
dtypes: int64(1), object(1)
memory usage: 16.9+ KB


,PubCHEM,drug_id
0,6035,Broxuridine
1,2435,Brimonidine
2,2333,Benzbromarone
3,16960,Dexbrompheniramine
4,2431,Bretylium
5,11626560,Crizotinib
6,5280489,Beta carotene
7,3955,Loperamide
8,1548953,Enclomiphene
9,6597,Dichloroacetic Acid


In [84]:
createFullTargetDF(cid_drug)

100%|██████████| 720/720 [27:21<00:00,  1.84s/it]


,cid,aid,score,entrezID,drug_id
0,6035,1458,14.1254,6607,Broxuridine
1,6035,1458,10,6607,Broxuridine
2,6035,1471,,3064,Broxuridine
3,6035,210689,5.06,7083,Broxuridine
4,6035,210691,25,7083,Broxuridine
5,6035,408386,5,888740,Broxuridine
6,6035,624032,4.0533,24660,Broxuridine
7,6035,624044,7.9373,24660,Broxuridine
8,6035,651751,39.8107,84385,Broxuridine
9,6035,651758,53.0804,3576,Broxuridine


In [95]:
##Make a data frame with one column called "PubCHEM" and other named "drug_id"
cid = {'PubCHEM': [x for x in combinedDF['PubChem Compound ID']], 'drug_id': [y for y in combinedDF['Name']]}
drug = pd.DataFrame(data=cid)


In [96]:
## same as before with less lines
drug = drug.dropna()

In [98]:
drug = drug.astype({"PubCHEM": int, "drug_id": object})

In [ ]:
drug = drug.astype(str)

In [101]:
createFullTargetDF(drug.head())

100%|██████████| 5/5 [00:07<00:00,  1.30s/it]


,cid,aid,score,entrezID,drug_id
0,6035,1458,14.1254,6607,Broxuridine
1,6035,1458,10,6607,Broxuridine
2,6035,1471,,3064,Broxuridine
3,6035,210689,5.06,7083,Broxuridine
4,6035,210691,25,7083,Broxuridine
5,6035,408386,5,888740,Broxuridine
6,6035,624032,4.0533,24660,Broxuridine
7,6035,624044,7.9373,24660,Broxuridine
8,6035,651751,39.8107,84385,Broxuridine
9,6035,651758,53.0804,3576,Broxuridine


In [165]:
## Load packages for model building
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import optimizers

In [93]:
## load dataframe with immunomodulatory effect scores
imscore = pd.read_excel("ImmunModPotential_Supp2.xlsx")

In [94]:
## column name didn't match from ImmunModPotential dataframe to our existing dataframe
imscore = imscore.rename(columns = {'name':"Name"})

In [166]:
## drug name didn't follow the same patter so change it to Capital letter + lower case and dropped null values
imscore.Name = imscore.Name.str.title()
imscore = imscore.dropna()

In [167]:
imscore.head()

,Name,VSV Infection,CD11C-CD11C,CD11C-CD14,CD11C-CD3,CD14-CD11C,CD14-CD14,CD14-CD19,CD19-CD14,CD19-CD19,CD19-CD3,CD3-CD11C,CD3-CD19,CD3-CD3
0,Bretylium Tosylate,0.281753,0.550864,0.074751,0.014859,0.290010,0.409782,0.560029,-0.402144,0.436481,-0.289821,-0.664059,0.232299,1.633227
1,Etomidate,0.292824,-0.965403,-1.736563,-0.367279,-0.124403,-0.993170,-0.206141,-0.192112,0.394961,-0.202929,-0.268586,-0.211872,-0.261213
2,Loperamide,-0.117341,-1.235752,0.227102,-0.182275,-0.985964,-0.438482,-0.509173,-0.592557,0.147891,-0.247796,0.067999,-0.998143,-0.166568
3,Chlorambucil,0.073640,0.088478,1.371637,-0.254888,0.778015,0.836844,-0.184801,-0.547305,-0.826591,-0.292145,-0.789511,-0.552819,-0.480118
4,Carmustine,-0.006240,-0.378749,0.125306,0.018359,-0.403721,-0.216324,-1.950322,-0.763955,-1.176017,-0.310338,0.386267,-0.270848,-0.251057


In [170]:
combinedDFF.Name = combinedDFF.Name.str.title()

In [171]:
## create a dataframe merging first dataframe(dataframe with mol calculation) with immunomodulatory potential score table by drug name
matrix = pd.merge(combinedDFF, imscore, on = "Name", how = "inner")

In [172]:
matrix.shape

(516, 1295)

In [181]:
## check columns with mol.cal calculation
matrix.iloc[:,25:1282].head()

,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,11.805899,0.0,0.0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,3.772584,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
1,12.216867,0.0,1.0,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,3.842414,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
2,18.449570,0.0,1.0,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,5.024165,...,10.344545,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778
3,18.449570,0.0,1.0,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,5.024165,...,10.344545,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778
4,20.542158,0.0,1.0,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,4.438453,...,10.600004,70.685296,476.223056,7.107807,3458.0,60.0,180.0,215.0,10.902778,7.486111


In [182]:
## For training datasets use columns containing molecular calculation columns
X = matrix.iloc[:,25:1282].values

In [183]:
## Check label column
matrix.iloc[:,1283:1284].head()

,CD11C-CD11C
0,0.460685
1,0.777075
2,2.043278
3,0.151665
4,-1.235752


In [184]:
## For label datasets use one column with CD11 - CD11 interaction column
Y = matrix.iloc[:,1283:1284]

In [185]:
## Create an artificial label based on score range. A = increasing immunesystem activity, B = low/no effect, C = decreasing immunesystem activity, U = unknown 
result = []
for i in Y['CD11C-CD11C']:
    if i >= 1:
        result.append("A")
    elif i >= -1:
        result.append("B")
    else:
        result.append("C")
print(len(result))

516


In [186]:
## add a column called "Label" with artificial labels created
Y.insert(loc=1, column='Label', value=result)

In [187]:
## Only get the values without the column name to use it as labeling set
Y = Y["Label"].values

In [188]:
## Change artificial label created with encoder changing letters into numbers ("A" = 0, "B" = 1, "C" = 2)
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
encoded_Y.shape

(516,)

In [189]:
## Set training datasets with label sets 
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size = 0.15)

In [160]:
X_train.shape

(464, 1257)

In [158]:
Y.shape

(489,)

In [190]:
model = Sequential()
model.add(Dense(2, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation ='sigmoid'))

In [194]:
keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [195]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [198]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
438/438 [==============================] - 0s 112us/sample - loss: nan - acc: 0.1393
Epoch 2/100
438/438 [==============================] - 0s 92us/sample - loss: nan - acc: 0.1393
Epoch 3/100
438/438 [==============================] - 0s 93us/sample - loss: nan - acc: 0.1393
Epoch 4/100
438/438 [==============================] - 0s 84us/sample - loss: nan - acc: 0.1393
Epoch 5/100
438/438 [==============================] - 0s 98us/sample - loss: nan - acc: 0.1393
Epoch 6/100
438/438 [==============================] - 0s 87us/sample - loss: nan - acc: 0.1393
Epoch 7/100
438/438 [==============================] - 0s 100us/sample - loss: nan - acc: 0.1393
Epoch 8/100
438/438 [==============================] - 0s 96us/sample - loss: nan - acc: 0.1393
Epoch 9/100
438/438 [==============================] - 0s 93us/sample - loss: nan - acc: 0.1393
Epoch 10/100
438/438 [==============================] - 0s 95us/sample - loss: nan - acc: 0.1393
Epoch 11/100
438/438 [===============

438/438 [==============================] - 0s 85us/sample - loss: nan - acc: 0.1393
Epoch 86/100
438/438 [==============================] - 0s 85us/sample - loss: nan - acc: 0.1393
Epoch 87/100
438/438 [==============================] - 0s 85us/sample - loss: nan - acc: 0.1393
Epoch 88/100
438/438 [==============================] - 0s 78us/sample - loss: nan - acc: 0.1393
Epoch 89/100
438/438 [==============================] - 0s 90us/sample - loss: nan - acc: 0.1393
Epoch 90/100
438/438 [==============================] - 0s 93us/sample - loss: nan - acc: 0.1393
Epoch 91/100
438/438 [==============================] - 0s 82us/sample - loss: nan - acc: 0.1393
Epoch 92/100
438/438 [==============================] - 0s 97us/sample - loss: nan - acc: 0.1393
Epoch 93/100
438/438 [==============================] - 0s 95us/sample - loss: nan - acc: 0.1393
Epoch 94/100
438/438 [==============================] - 0s 77us/sample - loss: nan - acc: 0.1393
Epoch 95/100
438/438 [=====================

In [199]:
_, accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %.2f' % (accuracy*100))

78/78 [==============================] - 0s 1ms/sample - loss: nan - acc: 0.1795
Accuracy: 17.95


In [200]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             multiple                  2516      
_________________________________________________________________
dense_29 (Dense)             multiple                  9         
_________________________________________________________________
dense_30 (Dense)             multiple                  12        
_________________________________________________________________
dense_31 (Dense)             multiple                  4         
Total params: 2,541
Trainable params: 2,541
Non-trainable params: 0
_________________________________________________________________
